In [1]:
from multiscaleio.datasets.get_arpa import get_arpa
from multiscaleio.core.expansions import ReshiftedExpansion, MultiscaleExpansion
import pandas as pd

In [2]:
arpa21 = get_arpa(n_jobs=5)

In [3]:
rs = ReshiftedExpansion(window=4, date_col="data", keep_t0=True)
data = rs.fit_transform(arpa21.drop(["lng", "lat", "provincia"], axis=1))

In [4]:
sh = pd.DataFrame(data, columns=rs.get_feature_names_out())
sh.head()

,data,Ammoniaca,Benzene,Biossido di Azoto,Biossido di Zolfo,BlackCarbon,Monossido di Azoto,Monossido di Carbonio,Ossidi di Azoto,Ozono,...,Benzene_shift_3,Biossido di Azoto_shift_3,Biossido di Zolfo_shift_3,BlackCarbon_shift_3,Monossido di Azoto_shift_3,Monossido di Carbonio_shift_3,Ossidi di Azoto_shift_3,Ozono_shift_3,PM10 (SM2005)_shift_3,Particelle sospese PM2.5_shift_3
0,2021-01-01,NaN,1.433333,28.519444,2.939583,NaN,NaN,0.763333,42.826389,23.928125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,NaN,1.0,28.232407,3.135417,NaN,NaN,0.665833,42.732407,30.742708,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,NaN,0.633333,18.002083,2.936458,NaN,NaN,0.581667,24.264236,44.576042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,NaN,1.033333,29.554282,3.445833,NaN,NaN,0.633333,48.063542,21.542708,...,1.433333,28.519444,2.939583,NaN,NaN,0.763333,42.826389,23.928125,43.0,36.4
4,2021-01-05,NaN,1.1,26.997454,3.465625,NaN,NaN,0.601667,44.580093,22.245833,...,1.0,28.232407,3.135417,NaN,NaN,0.665833,42.732407,30.742708,14.875,12.0


In [5]:
ms = MultiscaleExpansion(scale=[3, 7], date_col="data")
data = ms.fit_transform(sh)

In [6]:
sc = pd.DataFrame(data, columns=ms.get_feature_names_out())
sc.head()

,data,Ammoniaca_mean_3,Benzene_mean_3,Biossido di Azoto_mean_3,Biossido di Zolfo_mean_3,BlackCarbon_mean_3,Monossido di Azoto_mean_3,Monossido di Carbonio_mean_3,Ossidi di Azoto_mean_3,Ozono_mean_3,...,Benzene_shift_3_mean_7,Biossido di Azoto_shift_3_mean_7,Biossido di Zolfo_shift_3_mean_7,BlackCarbon_shift_3_mean_7,Monossido di Azoto_shift_3_mean_7,Monossido di Carbonio_shift_3_mean_7,Ossidi di Azoto_shift_3_mean_7,Ozono_shift_3_mean_7,PM10 (SM2005)_shift_3_mean_7,Particelle sospese PM2.5_shift_3_mean_7
0,2021-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,NaN,1.022222,24.917978,3.003819,NaN,NaN,0.670278,36.607677,33.082292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,NaN,0.888889,25.262924,3.172569,NaN,NaN,0.626944,38.353395,32.287153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-05,NaN,0.922222,24.851273,3.282639,NaN,NaN,0.605556,38.96929,29.454861,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
